# 简单的 RNN 文本生成

本 notebook 实现了一个基于字符级别的 RNN 模型，用于文本生成。

**主要内容：**
- 数据准备和预处理
- RNN 模型定义
- 训练过程
- 文本生成测试

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

## 1. 数据准备

使用简单的文本数据作为训练语料

In [ ]:
# 训练语料：简单的英文诗句
text = """
To be or not to be that is the question
Whether tis nobler in the mind to suffer
The slings and arrows of outrageous fortune
Or to take arms against a sea of troubles
And by opposing end them to die to sleep
No more and by a sleep to say we end
The heartache and the thousand natural shocks
That flesh is heir to tis a consummation
Devoutly to be wished to die to sleep
To sleep perchance to dream ay there is the rub
""".strip().lower()

print(f'文本长度: {len(text)} 个字符')
print(f'文本预览: {text[:80]}...')

In [ ]:
# 构建词汇表
chars = sorted(list(set(text)))
vocab_size = len(chars)

# 创建字符到索引和索引到字符的映射
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print(f'词汇表大小: {vocab_size}')
print(f'字符集: {"".join(chars)}')

In [ ]:
# 将文本编码为数字
data = [char_to_idx[ch] for ch in text]

# 准备训练数据：使用滑动窗口生成序列对
seq_length = 25  # 序列长度
X_data = []
y_data = []

for i in range(len(data) - seq_length):
    X_data.append(data[i:i+seq_length])
    y_data.append(data[i+1:i+seq_length+1])

X_data = np.array(X_data)
y_data = np.array(y_data)

print(f'训练样本数: {len(X_data)}')
print(f'输入形状: {X_data.shape}')
print(f'输出形状: {y_data.shape}')

## 2. 定义 RNN 模型

构建一个简单的字符级 RNN 模型

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(CharRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # RNN 层
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        
        # 全连接输出层
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden=None):
        # x: (batch_size, seq_length)
        batch_size = x.size(0)
        
        # 嵌入: (batch_size, seq_length, embedding_dim)
        embedded = self.embedding(x)
        
        # 初始化隐藏状态
        if hidden is None:
            hidden = self.init_hidden(batch_size)
        
        # RNN 前向传播
        rnn_out, hidden = self.rnn(embedded, hidden)
        # rnn_out: (batch_size, seq_length, hidden_dim)
        
        # 输出层
        output = self.fc(rnn_out)
        # output: (batch_size, seq_length, vocab_size)
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        # 初始化隐藏状态: (num_layers, batch_size, hidden_dim)
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)

# 模型超参数
embedding_dim = 50
hidden_dim = 128
num_layers = 2

# 创建模型
model = CharRNN(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
print(f'模型参数量: {sum(p.numel() for p in model.parameters())}')
print(model)

## 3. 训练模型

In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 训练参数
num_epochs = 500
batch_size = 32

# 将数据转换为 PyTorch 张量
X_train = torch.LongTensor(X_data).to(device)
y_train = torch.LongTensor(y_data).to(device)

# 训练循环
losses = []
model.train()

for epoch in range(num_epochs):
    # 随机打乱数据
    indices = torch.randperm(len(X_train))
    epoch_loss = 0
    
    # 按批次训练
    for i in range(0, len(X_train), batch_size):
        batch_indices = indices[i:i+batch_size]
        X_batch = X_train[batch_indices]
        y_batch = y_train[batch_indices]
        
        # 前向传播
        output, _ = model(X_batch)
        
        # 计算损失
        # output: (batch_size, seq_length, vocab_size)
        # y_batch: (batch_size, seq_length)
        loss = criterion(output.reshape(-1, vocab_size), y_batch.reshape(-1))
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # 记录平均损失
    avg_loss = epoch_loss / (len(X_train) // batch_size + 1)
    losses.append(avg_loss)
    
    # 每 50 个 epoch 打印一次
    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print('训练完成！')

In [ ]:
# 可视化损失曲线
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('训练损失曲线')
plt.grid(True)
plt.show()

## 4. 文本生成

使用训练好的模型生成新文本

In [ ]:
def generate_text(model, start_text, length=200, temperature=0.8):
    """
    生成文本
    
    参数:
        model: 训练好的模型
        start_text: 起始文本
        length: 生成的字符数
        temperature: 温度参数，控制随机性（值越大越随机）
    """
    model.eval()
    
    # 准备输入
    chars_generated = list(start_text.lower())
    input_seq = [char_to_idx[ch] for ch in start_text.lower()]
    
    # 初始化隐藏状态
    hidden = None
    
    with torch.no_grad():
        for _ in range(length):
            # 转换为张量
            x = torch.LongTensor([input_seq]).to(device)
            
            # 前向传播
            output, hidden = model(x, hidden)
            
            # 获取最后一个时间步的输出
            last_output = output[0, -1, :]
            
            # 应用温度
            last_output = last_output / temperature
            
            # 使用 softmax 获取概率分布
            probs = torch.softmax(last_output, dim=0).cpu().numpy()
            
            # 根据概率分布采样下一个字符
            next_char_idx = np.random.choice(len(probs), p=probs)
            next_char = idx_to_char[next_char_idx]
            
            # 添加到生成的文本
            chars_generated.append(next_char)
            
            # 更新输入序列（只保留最后 seq_length 个字符）
            input_seq.append(next_char_idx)
            input_seq = input_seq[-seq_length:]
    
    return ''.join(chars_generated)

print('文本生成函数已定义')

In [ ]:
# 测试 1: 从 "to be" 开始生成
print("=" * 50)
print("测试 1: 从 'to be' 开始生成文本")
print("=" * 50)
generated = generate_text(model, "to be", length=150, temperature=0.5)
print(generated)
print()

In [ ]:
# 测试 2: 从 "to sleep" 开始生成，使用更高温度（更随机）
print("=" * 50)
print("测试 2: 从 'to sleep' 开始生成文本（高温度，更随机）")
print("=" * 50)
generated = generate_text(model, "to sleep", length=150, temperature=1.0)
print(generated)
print()

In [ ]:
# 测试 3: 从 "the" 开始生成，使用低温度（更确定性）
print("=" * 50)
print("测试 3: 从 'the' 开始生成文本（低温度，更确定性）")
print("=" * 50)
generated = generate_text(model, "the", length=150, temperature=0.3)
print(generated)
print()

## 总结

### 模型架构
- **输入**: 字符序列（编码为整数）
- **嵌入层**: 将字符索引转换为稠密向量
- **RNN 层**: 2 层 RNN，隐藏维度 128
- **输出层**: 全连接层，输出词汇表大小的 logits

### 关键参数
- `seq_length=25`: 输入序列长度
- `embedding_dim=50`: 嵌入维度
- `hidden_dim=128`: RNN 隐藏层维度
- `num_layers=2`: RNN 层数
- `temperature`: 控制生成文本的随机性
  - 低温度（0.3）：更保守，倾向于选择高概率字符
  - 高温度（1.0）：更随机，增加多样性

### 改进方向
1. 使用更大的训练语料
2. 尝试 LSTM 或 GRU 替代简单 RNN
3. 增加模型层数和隐藏维度
4. 实现词级别（而非字符级别）的模型
5. 添加 dropout 防止过拟合
6. 使用学习率调度器